3.3. CIFAR-10 画像認識

CIFAR-10 は、カナダのトロント大学が提供する、10種類の画像を分類するためのデータセットです。このデータセットは、32x32ピクセルのカラー画像 60,000枚で構成されています。そのうち、50,000枚が訓練用、10,000枚がテスト用になっています。画像は、飛行機、自動車、鳥、猫、鹿、犬、カエル、馬、船、トラックの10種類です。

In [33]:
import os
import keras

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Flatten, Dense, Dropout

from keras.datasets import cifar10

from keras.optimizers import RMSprop

from keras.callbacks import TensorBoard, ModelCheckpoint


In [34]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [35]:
class CIFAR10Dataset():
  """
  CIFAR-10 dataset.
  attributes:
    - image_shape: image shape
    - num_classes: number of classes
  """
  def __init__(self):
    self.image_shape = (32, 32, 3)
    self.num_classes = 10

  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, self.num_classes)
    y_test = keras.utils.to_categorical(y_test, self.num_classes)

    return x_train, y_train, x_test, y_test
  
  def preprocess(slef, data, label_data=False):
    if label_data:
      data = keras.utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype('float32')
      data /= 255
      shape = (data.shape[0],) + self.image_shape
      return data

In [36]:
class Trainer():
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    self.verbose = 1
    self.log_dir = os.path.join(os.getcwd(), 'logdir')
    
    
    self.model_file_name = 'model_file.hdf5'

  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)

    self._target.fit(
      x_train, y_train,
      batch_size=batch_size, epochs=epochs,
      validation_split=validation_split,
      callbacks=[
        TensorBoard(log_dir=self.log_dir),
        ModelCheckpoint(os.path.join(self.log_dir, self.model_file_name), save_best_only=True)
      ],
      verbose=self.verbose)

In [37]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

170498071/170498071 [==============================] - 26s 0us/step


2024-02-02 10:45:56.553153: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 10:45:56.554711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 10:45:56.554736: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 10:45:56.557841: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-02 10:45:56.557861: I external/local_xla/xla/stream_executor

Epoch 1/12


2024-02-02 10:45:59.676155: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-02-02 10:46:00.803206: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-02 10:46:02.562755: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1de4028710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-02 10:46:02.562782: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-02-02 10:46:02.568600: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706838362.607428   28437 device_compiler.h:186] Compiled cluster using XL

313/313 [==============================] - 7s 6ms/step - loss: 1.7325 - accuracy: 0.3753 - val_loss: 1.3956 - val_accuracy: 0.5027
Epoch 2/12
 44/313 [===>..........................] - ETA: 0s - loss: 1.4411 - accuracy: 0.4846

/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 1s 4ms/step - loss: 1.3721 - accuracy: 0.5099 - val_loss: 1.2178 - val_accuracy: 0.5807
Epoch 3/12
313/313 [==============================] - 1s 4ms/step - loss: 1.2174 - accuracy: 0.5726 - val_loss: 1.1853 - val_accuracy: 0.5974
Epoch 4/12
313/313 [==============================] - 1s 5ms/step - loss: 1.1154 - accuracy: 0.6071 - val_loss: 1.0855 - val_accuracy: 0.6254
Epoch 5/12
313/313 [==============================] - 1s 5ms/step - loss: 1.0397 - accuracy: 0.6324 - val_loss: 0.9932 - val_accuracy: 0.6484
Epoch 6/12
313/313 [==============================] - 1s 4ms/step - loss: 0.9803 - accuracy: 0.6563 - val_loss: 1.1114 - val_accuracy: 0.6037
Epoch 7/12
313/313 [==============================] - 1s 5ms/step - loss: 0.9362 - accuracy: 0.6720 - val_loss: 0.9068 - val_accuracy: 0.6857
Epoch 8/12
313/313 [==============================] - 1s 4ms/step - loss: 0.8946 - accuracy: 0.6867 - val_loss: 0.9618 - val_accuracy: 0.6653
Epoch 9/12
313/31

参考書と同じコードだと思うが、結果が大違いだ！

In [38]:
# check tensorboard
# !tensorboard --logdir=logdir

**ネットワークをもっと深くすることによる改善**

In [39]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [40]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
313/313 [==============================] - 3s 6ms/step - loss: 1.8644 - accuracy: 0.3211 - val_loss: 1.4696 - val_accuracy: 0.4587
Epoch 2/12
313/313 [==============================] - 2s 5ms/step - loss: 1.4342 - accuracy: 0.4867 - val_loss: 1.2785 - val_accuracy: 0.5337
Epoch 3/12
313/313 [==============================] - 2s 5ms/step - loss: 1.2290 - accuracy: 0.5642 - val_loss: 1.0850 - val_accuracy: 0.6102
Epoch 4/12
313/313 [==============================] - 2s 5ms/step - loss: 1.1114 - accuracy: 0.6045 - val_loss: 0.9798 - val_accuracy: 0.6455
Epoch 5/12
313/313 [==============================] - 2s 5ms/step - loss: 1.0154 - accuracy: 0.6425 - val_loss: 1.1113 - val_accuracy: 0.6253
Epoch 6/12
313/313 [==============================] - 2s 5ms/step - loss: 0.9447 - accuracy: 0.6698 - val_loss: 0.9162 - val_accuracy: 0.6712
Epoch 7/12
313/313 [==============================] - 2s 5ms/step - loss: 0.8891 - accuracy: 0.6864 - val_loss: 0.7770 - val_accuracy: 0.7311
Epoch 